In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# base_path = ""
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
submission = pd.read_csv('./data/gender_submission.csv')

In [3]:
# 1. 결측치 체크
train.info() # Age, Cabin, Embarked
test.info() # Age, Fare, Cabin

# 2. dtype이 Object인 column 찾기
# train.columns[train.dtypes == 'object']

# 3. target value distribution 체크: class별 비율 차이가 큰지 확인 (10:1 이상인지 확인)
# train.Survived.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass  

In [4]:
# feature engineering (데이터 가공 -> How?)
# 1. Cabin column의 결측치 여부가 Survived에 영향이 있는가? (보는 이유 -> Cabin이 결측치가 많아서 -> Cabin이 없어도 정보가 될까?)
# print(train.loc[train.Cabin.isnull(), 'Survived'].sum())
# print(train.loc[train.Cabin.notnull(), 'Survived'].sum())
# print(train.loc[train.Cabin.isnull(), 'Survived'].mean(), train.loc[train.Cabin.notnull(), 'Survived'].mean()) # Cabin의 결측치 여부가 Survived에 영향을 미칠 수도 있음 -> why? -> 수치적으로 차이가 있음

# 2. 영향이 있는것 같으니, Cabin값이 있으면 1, 없으면 0 값을 가지는 is_Cabin이라는 column을 만들어봅시다.
# train.Cabin.notnull() # 값이 있으면 = False면 1, 값이 없으면 = True면 0
train['is_Cabin'] = train.Cabin.notnull() * 1
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,is_Cabin
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0


In [5]:
# 3. SibSp + Parch = FamilySize
train['FamilySize'] = train.SibSp + train.Parch # 이때 파생변수만 남기고, 기존 2개의 컬럼을 drop할지도 정해야 함 -> why? -> 다중공선성을 가질 수 있기 때문에 -> x1 + x2 = x3
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,is_Cabin,FamilySize
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,0


In [6]:
# 4. drop columns
drop_cols = ['Cabin', 'SibSp', 'Parch', 'PassengerId', 'Name', 'Ticket']
train = train.drop(columns=drop_cols)
train.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,is_Cabin,FamilySize
0,0,3,male,22.0,7.2500,S,0,1
1,1,1,female,38.0,71.2833,C,1,1
2,1,3,female,26.0,7.9250,S,0,0
3,1,1,female,35.0,53.1000,S,1,1
4,0,3,male,35.0,8.0500,S,0,0


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Survived    891 non-null    int64  
 1   Pclass      891 non-null    int64  
 2   Sex         891 non-null    object 
 3   Age         714 non-null    float64
 4   Fare        891 non-null    float64
 5   Embarked    889 non-null    object 
 6   is_Cabin    891 non-null    int32  
 7   FamilySize  891 non-null    int64  
dtypes: float64(2), int32(1), int64(3), object(2)
memory usage: 52.3+ KB


In [8]:
# 5. Age column (drop 후 Age에 많은 결측치가 존재함)
# train.Age.describe()

# missing value 처리 방법
# from sklearn.impute import KNNImputer

train.Age = train.Age.fillna(train.Age.median())

In [9]:
# 6. dropna (Embarked는 결측치가 작아서 일단 dropna)
train = train.dropna()

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Survived    889 non-null    int64  
 1   Pclass      889 non-null    int64  
 2   Sex         889 non-null    object 
 3   Age         889 non-null    float64
 4   Fare        889 non-null    float64
 5   Embarked    889 non-null    object 
 6   is_Cabin    889 non-null    int32  
 7   FamilySize  889 non-null    int64  
dtypes: float64(2), int32(1), int64(3), object(2)
memory usage: 59.0+ KB


In [11]:
# 7. Encoding categorical feature
# One-hot encoding (컬럼 개수가 안 많아지도록 유의하기)
train = pd.get_dummies(data=train, columns=['Sex', 'Embarked'])
train.head()

,Survived,Pclass,Age,Fare,is_Cabin,FamilySize,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.0,7.2500,0,1,0,1,0,0,1
1,1,1,38.0,71.2833,1,1,1,0,1,0,0
2,1,3,26.0,7.9250,0,0,1,0,0,0,1
3,1,1,35.0,53.1000,1,1,1,0,0,0,1
4,0,3,35.0,8.0500,0,0,0,1,0,0,1


In [12]:
# data setting for training
from sklearn.model_selection import train_test_split

X = train.drop(columns='Survived')
y = train.Survived

# 트테트테
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(711, 10) (178, 10) (711,) (178,)


In [13]:
# sklearn에서 학습을 하는 방법
# 1. 사용할 함수를 불러온다.
from sklearn.linear_model import SGDClassifier  # Linear Classifier
from sklearn.tree import DecisionTreeClassifier # CART
from sklearn.ensemble import RandomForestClassifier

# 2. 해당 함수를 선언한다.
# clf = SGDClassifier()
# clf2 = DecisionTreeClassifier()
clf3 = RandomForestClassifier(
    max_depth=5,
    n_estimators=500,
    max_features=0.9,
    min_samples_split=5,
    random_state=42,
)

# 3. fit()
# clf.fit(X_train, y_train)
# clf2.fit(X_train, y_train)
clf3.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, max_features=0.9, min_samples_split=5,
                       n_estimators=500, random_state=42)

In [14]:
from sklearn.metrics import accuracy_score

# accuracy_score(y_train, clf.predict(X_train)) # P_train = 0.71 (underfitting -> 폐기)
# print('Train ACC: %.4f' % accuracy_score(y_train, clf2.predict(X_train)))
# print('Valid ACC: %.4f' % accuracy_score(y_val, clf2.predict(X_val)))
print('Train ACC: %.4f' % accuracy_score(y_train, clf3.predict(X_train))) # RF 선정! Train ACC: 0.9873, Valid ACC: 0.7753
print('Valid ACC: %.4f' % accuracy_score(y_val, clf3.predict(X_val)))

Train ACC: 0.8678
Valid ACC: 0.8146


In [15]:
## TO-DO: test 데이터를 X_test로 가공해서 clf3.predict(X_test)가 에러가 나지않고 출력이 되게끔 만드세요!
test['is_Cabin'] = test.Cabin.notnull() * 1
test['FamilySize'] = test.SibSp + test.Parch
test = test.drop(columns=drop_cols)
test.Age = test.Age.fillna(test.Age.median())

In [16]:
test[test.Fare.isnull()]

,Pclass,Sex,Age,Fare,Embarked,is_Cabin,FamilySize
152,3,male,60.5,NaN,S,0,0


In [26]:
# test = test.dropna()
a = train.loc[(60 <= train.Age) & (train.Pclass == 3) & (train.is_Cabin == 0) & (train.FamilySize == 0) & (train.Sex_male == 1) & (train.Embarked_S == 1)].Fare.mean()
test = test.fillna(a)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      418 non-null    int64  
 1   Sex         418 non-null    object 
 2   Age         418 non-null    float64
 3   Fare        418 non-null    float64
 4   Embarked    418 non-null    object 
 5   is_Cabin    418 non-null    int32  
 6   FamilySize  418 non-null    int64  
dtypes: float64(2), int32(1), int64(2), object(2)
memory usage: 21.4+ KB


In [27]:
test = pd.get_dummies(data=test, columns=['Sex', 'Embarked'])
test.head()

,Pclass,Age,Fare,is_Cabin,FamilySize,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,34.5,7.8292,0,0,0,1,0,1,0
1,3,47.0,7.0000,0,1,1,0,0,0,1
2,2,62.0,9.6875,0,0,0,1,0,1,0
3,3,27.0,8.6625,0,0,0,1,0,0,1
4,3,22.0,12.2875,0,2,1,0,0,0,1


In [28]:
preds = clf3.predict(test)
preds

array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [31]:
submission['Survived'] = preds
submission.to_csv('./rf_0.csv', index=False)